In [1]:
import sys
sys.path.append('..')
import utils

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

In [2]:
from mxnet import gluon

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(10))
net.initialize()

In [3]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [4]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [9]:
from mxnet import ndarray as nd
from mxnet import autograd as ag

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with ag.record():
            out = net(data)
            loss = softmax_cross_entropy(out, label)
        loss.backward()
        trainer.step(batch_size)

        label = label.astype('float32')
        #print((out.argmax(axis=1)==label))
        #print(out.argmax(axis=1))
        #print(label)
        #print((out.argmax(axis=1)==label).sum().asscalar())
        #print(nd.mean(out.argmax(axis=1)==label).asscalar())
        #break
        train_loss += nd.mean(loss).asscalar()
        #train_acc += utils.accuracy(out, label)
        train_acc += nd.mean(out.argmax(axis=1)==label).asscalar()

        
    test_acc = utils.evaluate_accuracy(test_data, net)
    print('epoch %d. Loss: %f, Train acc %f, Test acc %f' % (
        epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

epoch 0. Loss: 0.478838, Train acc 0.838453, Test acc 0.842300
epoch 1. Loss: 0.470889, Train acc 0.839445, Test acc 0.839000
epoch 2. Loss: 0.462779, Train acc 0.842127, Test acc 0.846600
epoch 3. Loss: 0.457589, Train acc 0.842692, Test acc 0.848300
epoch 4. Loss: 0.452063, Train acc 0.845324, Test acc 0.844000
